In [2]:
import pymatgen.core as pmg
import numpy as np

First thing we generate the bulk $\mathrm{fcc}$ structure. 
* we define the lattice with the 3 fcc vectors
* a list with a symbol for the atoms in the structure ( in this just one Al) 
* the coordinates in the same order as the symbols is a list of np.arrays of length 3

In [3]:
alat = 10.21 
lattice = np.array([[0.,1.,1.], [1.,0.,1], [1.,1.,0]])* alat *0.5
species = ['Si','Si']
coords=[np.array([0,0,0]), np.array([0.25,0.25,0.25])]

s = pmg.Structure(lattice=lattice, species=species , coords=coords, coords_are_cartesian=False) 

just to check everything is fine we make print out the info about the structure

In [4]:
s

Structure Summary
Lattice
    abc : 7.219560235914651 7.219560235914651 7.219560235914651
 angles : 60.00000000000001 60.00000000000001 60.00000000000001
 volume : 266.08306525000006
      A : 0.0 5.105 5.105
      B : 5.105 0.0 5.105
      C : 5.105 5.105 0.0
    pbc : True True True
PeriodicSite: Si (0.0000, 0.0000, 0.0000) [0.0000, 0.0000, 0.0000]
PeriodicSite: Si (2.5525, 2.5525, 2.5525) [0.2500, 0.2500, 0.2500]

we now generate a supercell with the $z$  axis oriented along the $[1 0 0]$ direction and the other 2 axes perpendicular to it, generating the 4X2 conventional cell  
* we copy the bulk structure into s1 
* we transform `sup1` in a supercell. The  input in the second line means that the new vectors of the lattice for the supercell are $$\bf{a_1^\prime} = -\bf{a_1} + -2\bf{a_2}+2\bf{a_3}  \; ; \; \bf{a_2^\prime} = \bf{a_1}-2\bf{a_2}+2\bf{a_3} \: ; \: \bf{a_3^\prime} = -\bf{a_1} + \bf{a_2} +\bf{a_3}  $$

In [5]:
sup1 = s.copy()
sup1.make_supercell([[-1,-2,2],[1,-2,2],[-1,1,1]], to_unit_cell = True)
sup1, sup1.lattice.matrix

(Structure Summary
 Lattice
     abc : 16.14342745515958 16.14342745515958 10.21
  angles : 90.0 90.0 53.13010235415598
  volume : 2128.6645220000005
       A : 0.0 5.105 -15.315000000000001
       B : 0.0 15.315000000000001 -5.105
       C : 10.21 0.0 0.0
     pbc : True True True
 PeriodicSite: Si (5.1050, 10.2100, -15.3150) [0.8750, 0.3750, 0.5000]
 PeriodicSite: Si (5.1050, 5.1050, -10.2100) [0.6250, 0.1250, 0.5000]
 PeriodicSite: Si (0.0000, 15.3150, -15.3150) [0.7500, 0.7500, 0.0000]
 PeriodicSite: Si (0.0000, 10.2100, -10.2100) [0.5000, 0.5000, 0.0000]
 PeriodicSite: Si (5.1050, 15.3150, -10.2100) [0.3750, 0.8750, 0.5000]
 PeriodicSite: Si (0.0000, 5.1050, -5.1050) [0.2500, 0.2500, 0.0000]
 PeriodicSite: Si (5.1050, 10.2100, -5.1050) [0.1250, 0.6250, 0.5000]
 PeriodicSite: Si (0.0000, 0.0000, 0.0000) [0.0000, 0.0000, 0.0000]
 PeriodicSite: Si (7.6575, 12.7625, -12.7625) [0.6250, 0.6250, 0.7500]
 PeriodicSite: Si (7.6575, 7.6575, -7.6575) [0.3750, 0.3750, 0.7500]
 PeriodicSite: S

Now we need to transform the axes in order that $\bf{c}$ points in the $z$ direction.
The object `lattice` of pymatgen has all the info that we need. The 3 axis lengths $a,b,c$ and the angles between the axes $\alpha$ between b and c axes , $\beta$ between $\bf{a}$  and $\bf{c}$ and $\gamma$ between $\bf{a}$ and $\bf{b}$. 

We just set ${\bf c}^\prime = c \cdot \hat{\bf z}$; we then put ${\bf a}^\prime = a\cdot sin(\beta)\cdot \hat{\bf x} + a\cdot cos(\beta)\cdot\hat{\bf z}$ in the $xz$ plane forming an angle $\beta$ with ${\bf c}^\prime$; the components of $\mathbf{b}^\prime$ are found imposing that ${\bf b^\prime \cdot c^\prime} = b\,c\cdot cos(\alpha)$ and 
${\bf b^\prime \cdot a^\prime} = a\,b \cdot cos(\gamma)$

Just use this numbers to create the new vectors and print them to check we have done right

In [6]:
latt = sup1.lattice
a = latt.a ; alpha = latt.alpha*np.pi/180.0
b = latt.b ; beta = latt.beta*np.pi/180.0
c = latt.c ; gamma = latt.gamma *np.pi/180.0
print(a,b,c, alpha/np.pi*180, beta/np.pi*180, gamma/np.pi*180)
newa3 = c * np.array([0,0,1])
newa1 = a * np.array([np.sin(beta),0,np.cos(beta)])
ca = (np.cos(gamma) - np.cos(beta)*np.cos(alpha))/np.sin(beta)
cb = np.sqrt(np.sin(alpha)**2-ca**2)
newa2 = b * np.array([ca,cb,np.cos(alpha)])
newa1,newa2,newa3

16.14342745515958 16.14342745515958 10.21 90.0 90.0 53.13010235415599


(array([1.61434275e+01, 0.00000000e+00, 9.88499838e-16]),
 array([9.68605647e+00, 1.29147420e+01, 9.88499838e-16]),
 array([ 0.  ,  0.  , 10.21]))


we rotate the new $\bf{a_1}$ and $\bf{a_2}$ in the plase so that these  axes are oriented compatibly with `pw.x` for the orthorhombic base-centered case with `ibrav=-9` 

``` 
(a/2,-b/2,0),(a/2,b/2,0),(0,0,c)
```

In [7]:
bisect = (newa1 + newa2)/np.sqrt((newa1+newa2).dot(newa1+newa2)) 
cbisect = newa1.dot(bisect) / a 
newa1 = a * np.array([cbisect, - np.sqrt(1 - cbisect * cbisect), 0.0]) 
newa2 = a * np.array([cbisect,   np.sqrt(1 - cbisect * cbisect), 0.0])
newA = 4 * np.sqrt(1./2) * alat 
newB = 2 * np.sqrt(1./2) * alat 
newa1, newa2,newa3, newA/2., newB/2. 

(array([14.43912047, -7.21956024,  0.        ]),
 array([14.43912047,  7.21956024,  0.        ]),
 array([ 0.  ,  0.  , 10.21]),
 14.439120471829302,
 7.219560235914651)

now we replace these three axes in the structure.  then we print out structure info  to compare lengths, angles and fractional coordinates. 

In [8]:
sup1.lattice = pmg.Lattice(np.array([newa1, newa2,newa3]))
sup1, sup1.lattice

(Structure Summary
 Lattice
     abc : 16.14342745515958 16.14342745515958 10.21
  angles : 90.0 90.0 53.130102354155994
  volume : 2128.664522000001
       A : 14.439120471829302 -7.219560235914653 0.0
       B : 14.439120471829302 7.219560235914653 0.0
       C : 0.0 0.0 10.21
     pbc : True True True
 PeriodicSite: Si (18.0489, -3.6098, 5.1050) [0.8750, 0.3750, 0.5000]
 PeriodicSite: Si (10.8293, -3.6098, 5.1050) [0.6250, 0.1250, 0.5000]
 PeriodicSite: Si (21.6587, 0.0000, 0.0000) [0.7500, 0.7500, 0.0000]
 PeriodicSite: Si (14.4391, 0.0000, 0.0000) [0.5000, 0.5000, 0.0000]
 PeriodicSite: Si (18.0489, 3.6098, 5.1050) [0.3750, 0.8750, 0.5000]
 PeriodicSite: Si (7.2196, 0.0000, 0.0000) [0.2500, 0.2500, 0.0000]
 PeriodicSite: Si (10.8293, 3.6098, 5.1050) [0.1250, 0.6250, 0.5000]
 PeriodicSite: Si (0.0000, 0.0000, 0.0000) [0.0000, 0.0000, 0.0000]
 PeriodicSite: Si (18.0489, 0.0000, 7.6575) [0.6250, 0.6250, 0.7500]
 PeriodicSite: Si (10.8293, 0.0000, 7.6575) [0.3750, 0.3750, 0.7500]
 Per

Now that the c vector is  oriented along the z cartesian axis we can build any supercell along the $[100]$ direction of any the periodicity along the xy  plane and of any thickess along the z direction 

the first two vectors define the periodicity in the plane , the third vector the thickness. For example a $2\times2$ surface with thickness 3 is generated like this:

* first we generate the supercell 

In [9]:
sup2 = sup1.copy()
sup2.make_supercell([[1,0,0],[0,1,0],[0,0,2]], to_unit_cell = False)
sup2

Structure Summary
Lattice
    abc : 16.14342745515958 16.14342745515958 20.42
 angles : 90.0 90.0 53.130102354155994
 volume : 4257.329044000002
      A : 14.439120471829302 -7.219560235914653 0.0
      B : 14.439120471829302 7.219560235914653 0.0
      C : 0.0 0.0 20.42
    pbc : True True True
PeriodicSite: Si (18.0489, -3.6098, 5.1050) [0.8750, 0.3750, 0.2500]
PeriodicSite: Si (18.0489, -3.6098, 15.3150) [0.8750, 0.3750, 0.7500]
PeriodicSite: Si (10.8293, -3.6098, 5.1050) [0.6250, 0.1250, 0.2500]
PeriodicSite: Si (10.8293, -3.6098, 15.3150) [0.6250, 0.1250, 0.7500]
PeriodicSite: Si (21.6587, -0.0000, 0.0000) [0.7500, 0.7500, 0.0000]
PeriodicSite: Si (21.6587, -0.0000, 10.2100) [0.7500, 0.7500, 0.5000]
PeriodicSite: Si (14.4391, -0.0000, 0.0000) [0.5000, 0.5000, 0.0000]
PeriodicSite: Si (14.4391, -0.0000, 10.2100) [0.5000, 0.5000, 0.5000]
PeriodicSite: Si (18.0489, 3.6098, 5.1050) [0.3750, 0.8750, 0.2500]
PeriodicSite: Si (18.0489, 3.6098, 15.3150) [0.3750, 0.8750, 0.7500]
PeriodicSi

* then we extract the coordinates and order them along the z direction, in case we have different species we need to sort also the symbols .... 

In [10]:
coords = sup2.cart_coords
species = sup2.species
temp  = sorted(zip(species,coords), key= lambda x: x[1][2])
species = [_[0] for _ in temp ]
coords  = [_[1] for _ in temp ]
coords

[array([ 2.16586807e+01, -8.88178420e-16,  0.00000000e+00]),
 array([ 1.44391205e+01, -4.44089210e-16,  0.00000000e+00]),
 array([ 7.21956024e+00, -2.22044605e-16,  0.00000000e+00]),
 array([0., 0., 0.]),
 array([21.65868071,  3.60978012,  2.5525    ]),
 array([14.43912047,  3.60978012,  2.5525    ]),
 array([7.21956024, 3.60978012, 2.5525    ]),
 array([14.43912047, -3.60978012,  2.5525    ]),
 array([18.04890059, -3.60978012,  5.105     ]),
 array([10.82934035, -3.60978012,  5.105     ]),
 array([18.04890059,  3.60978012,  5.105     ]),
 array([10.82934035,  3.60978012,  5.105     ]),
 array([ 1.80489006e+01, -8.88178420e-16,  7.65750000e+00]),
 array([ 1.08293404e+01, -4.44089210e-16,  7.65750000e+00]),
 array([ 3.60978012e+00, -1.66533454e-15,  7.65750000e+00]),
 array([ 2.52684608e+01, -8.88178420e-16,  7.65750000e+00]),
 array([ 2.16586807e+01, -8.88178420e-16,  1.02100000e+01]),
 array([ 1.44391205e+01, -4.44089210e-16,  1.02100000e+01]),
 array([ 7.21956024e+00, -2.22044605e-16

* then we modify the periodicit along the a3 axes. We replace previous $\bf{c}$ vector with one that is longer than the lattice periodity in that direction this create a gap between the slabs. 

In [11]:
a1 = sup2.lattice.matrix[0]
a2 = sup2.lattice.matrix[1]
vacuum = 10 # length of the vacuum region 
a3 = sup2.lattice.matrix[2]+np.array([0.,0., vacuum])
surface = pmg.Structure(lattice = np.array([a1,a2,a3]), coords =coords, species= species, coords_are_cartesian=True)
from	collections import deque
it = (f"{_.specie.name}   {_.coords[0]:12.6f}  {_.coords[1]:12.6f}  {_.coords[2]:12.6f} \n" for _ in surface.sites) 
celldm1 = newA 
celldm2 = newB/newA 
celldm3 = np.sqrt(surface.lattice.matrix[2].dot(surface.lattice.matrix[2]))/celldm1  
with open('posizioniSi100_8layer','w') as f:
	f.write(f"ibrav=-9    celldm(1)={celldm1:9.6f}   celldm(2)={celldm2:9.6f}   celldm(3)={celldm3:9.6f}" +
	        f"   nat={len(surface.sites)}\n\n")
	s = [" ".join((f"{round(_,6):12.6}" for _ in surface.lattice.matrix[i])) for i in [0,1,2]]    
	f.write ( "\n".join(s) + "\n\n")
	deque((f.write(_) for _ in it )) 
 

In [22]:
surfcopy = surface.copy()
surfcopy.lattice   = pmg.Lattice(surface.lattice.matrix*0.529177)
surfcopy.to(filename='surf.cif')

In [13]:
p = surface.sites[4]

In [14]:
surface.lattice.matrix

array([[14.43912047, -7.21956024,  0.        ],
       [14.43912047,  7.21956024,  0.        ],
       [ 0.        ,  0.        , 30.42      ]])

In [15]:
a = surface.lattice.matrix 
print (f"alat = { np.sqrt(a[0].dot(a[0]))} cosgamma={a[0].dot(a[1])/a[1].dot(a[1])} ")

alat = 16.14342745515958 cosgamma=0.5999999999999999 


In [16]:
np.arccos(0.6)/np.pi**2

0.093954649073803

In [30]:
crystal_k_points = [
	[ 0.0000,  0.0000, 0.0000],
	[ 0.5000,  0.5000, 0.0000],
	[ 0.3125,  0.6875, 0.0000],
	[-0.3125,  0.3125, 0.0000],
	[ 0.0000,  0.0000, 0.0000]
] 
crystal_k_points = np.array(crystal_k_points)
from bravais import recip 

b = recip(surface.lattice.matrix)
np.round(crystal_k_points.dot(b)/2/np.pi*celldm1,5) 


array([[0.  , 0.  , 0.  ],
       [1.  , 0.  , 0.  ],
       [1.  , 0.75, 0.  ],
       [0.  , 1.25, 0.  ],
       [0.  , 0.  , 0.  ]])